In [1]:
import sys, os
import time, random, pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('feature_set/poses_data_all.csv')
len(df)

5

In [3]:
df.Lin_F9

0    5.773
1    5.716
2    5.666
3    5.559
4    5.541
Name: Lin_F9, dtype: float64

In [4]:
LigandDescriptors = ['HeavyAtomMolWt', 'NumValenceElectrons','FpDensityMorgan1', 
                     'FpDensityMorgan2', 'FpDensityMorgan3', 'LabuteASA',
                     'TPSA', 'NHOHCount', 'MolLogP','MolMR'] #

In [5]:
f_type = ["P","N","DA","D","A","AR","H","PL","HA","SA"]
SASA = ["P2." + i for i in f_type] + ["P2dl." + i for i in f_type] + ["P2dp." + i for i in f_type]
SASA = [x for x in SASA if x not in ['P2dl.HA','P2dp.HA']]

vina1 = ['vina%d'%x for x in range(2,9)]
vina2 = ['vina%d'%x for x in range(10,17)]
vina3 = ['vina%d'%x for x in range(18,25)]
vina4 = ['vina%d'%x for x in range(26,31)]
vina5 = ['vina%d'%x for x in range(32,37)]
vina6 = ['vina%d'%x for x in range(38,49)]

vina=vina1+vina2+vina3+vina4+vina5+vina6
metal = ['metal%d'%x for x in range(2,8)]
BW = ["Nbw","Epw","Elw"]

columns = vina+['lig_Cover','betaScore','LE']+SASA+metal+LigandDescriptors+BW
#df['LE'] = df['Lin_F9']/df['vina45']
len(columns)
len(columns)

92

In [6]:
X = np.c_[df[columns]]
X = X.astype(np.float64)
y_fix = np.r_[df['Lin_F9']]

In [7]:
df['Lin_F9']

0    5.773
1    5.716
2    5.666
3    5.559
4    5.541
Name: Lin_F9, dtype: float64

In [8]:
df['LE']

0    0.180406
1    0.178625
2    0.177063
3    0.173719
4    0.173156
Name: LE, dtype: float64

In [11]:
y_predict_ = []
for i in range(1,11):
    xgb_model = pickle.load(open("/Users/zixuanfeng/Desktop/computational/HSP70s_family_inhibitors_docking/XGB_model/mod_12_%d.pickle.dat"%i,"rb"))
    y_i_predict = xgb_model.predict(X, ntree_limit=xgb_model.best_ntree_limit)
    y_predict_.append(y_i_predict)

y_predict = np.average(y_predict_, axis=0)

In [12]:
y_predict

array([-0.31431377, -0.24143088, -0.27442914,  0.04560479, -0.1341627 ],
      dtype=float32)

In [13]:
df['XGB'] = pd.Series(y_predict+y_fix, index=df.index)

In [14]:
df

,ligand_id,SMILES,compounds_name,Pubchem_cid,Lin_F9_Score,o_index,betaScore,lig_Cover,Lin_F9,HeavyAtomMolWt,...,P2dp.DA,P2dp.D,P2dp.A,P2dp.AR,P2dp.H,P2dp.PL,P2dp.HA,P2dp.SA,LE,XGB
0,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.870110,1,-1.5,0.281,5.773,513.391,...,0,0,26.71,19.29,0,23.00,0,84.26,0.180406,5.458686
1,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.792520,2,-5.8,0.594,5.716,513.391,...,0,0,0.63,0.00,0,19.57,0,20.20,0.178625,5.474569
2,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.724721,3,-7.4,0.781,5.666,513.391,...,0,0,1.05,0.00,0,28.54,0,29.59,0.177063,5.391571
3,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.579110,4,-6.6,0.719,5.559,513.391,...,0,0,164.78,109.91,0,92.55,0,583.94,0.173719,5.604605
4,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.553894,5,-8.0,0.812,5.541,513.391,...,0,0,0.91,0.00,0,22.32,0,23.23,0.173156,5.406837


In [15]:
df = df[['ligand_id','o_index','SMILES','compounds_name','Pubchem_cid','betaScore','lig_Cover','vina45','Lin_F9_Score','LE','XGB']]

In [16]:
df.head(2)

,ligand_id,o_index,SMILES,compounds_name,Pubchem_cid,betaScore,lig_Cover,vina45,Lin_F9_Score,LE,XGB
0,1,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-1.5,0.281,32.0,-7.87011,0.180406,5.458686
1,1,2,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-5.8,0.594,32.0,-7.79252,0.178625,5.474569


In [17]:
df['XGB_score'] = round(df['XGB']/(-0.73349),3)

/Users/zixuanfeng/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df['XGB_score_LE'] = round(df['XGB_score']/df['vina45'],3)

/Users/zixuanfeng/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
df['LinF9_score_LE'] = round(df['Lin_F9_Score']/df['vina45'],3)

/Users/zixuanfeng/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df

,ligand_id,o_index,SMILES,compounds_name,Pubchem_cid,betaScore,lig_Cover,vina45,Lin_F9_Score,LE,XGB,XGB_score,XGB_score_LE,LinF9_score_LE
0,1,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-1.5,0.281,32.0,-7.870110,0.180406,5.458686,-7.442,-0.233,-0.246
1,1,2,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-5.8,0.594,32.0,-7.792520,0.178625,5.474569,-7.464,-0.233,-0.244
2,1,3,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.4,0.781,32.0,-7.724721,0.177063,5.391571,-7.351,-0.230,-0.241
3,1,4,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-6.6,0.719,32.0,-7.579110,0.173719,5.604605,-7.641,-0.239,-0.237
4,1,5,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-8.0,0.812,32.0,-7.553894,0.173156,5.406837,-7.371,-0.230,-0.236


In [21]:
df=df[['ligand_id','o_index','SMILES','compounds_name','Pubchem_cid','betaScore','lig_Cover','vina45','Lin_F9_Score','XGB','XGB_score','XGB_score_LE']]

In [22]:
df

,ligand_id,o_index,SMILES,compounds_name,Pubchem_cid,betaScore,lig_Cover,vina45,Lin_F9_Score,XGB,XGB_score,XGB_score_LE
0,1,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-1.5,0.281,32.0,-7.870110,5.458686,-7.442,-0.233
1,1,2,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-5.8,0.594,32.0,-7.792520,5.474569,-7.464,-0.233
2,1,3,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-7.4,0.781,32.0,-7.724721,5.391571,-7.351,-0.230
3,1,4,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-6.6,0.719,32.0,-7.579110,5.604605,-7.641,-0.239
4,1,5,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-8.0,0.812,32.0,-7.553894,5.406837,-7.371,-0.230


In [25]:
df.to_csv('final_set/site1.csv')

In [26]:
df.head(2)

,ligand_id,o_index,SMILES,compounds_name,Pubchem_cid,betaScore,lig_Cover,vina45,Lin_F9_Score,XGB,XGB_score,XGB_score_LE
0,1,1,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-1.5,0.281,32.0,-7.87011,5.458686,-7.442,-0.233
1,1,2,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-5.8,0.594,32.0,-7.79252,5.474569,-7.464,-0.233


###check XGB_score

In [27]:
df = pd.read_csv('final_set/site1.csv')


In [28]:
df1=df.sort_values(by=['ligand_id','XGB'], ascending=[True,False])
df1 = df1.drop_duplicates('ligand_id')
len(df1)

1

In [29]:
df1

,Unnamed: 0,ligand_id,o_index,SMILES,compounds_name,Pubchem_cid,betaScore,lig_Cover,vina45,Lin_F9_Score,XGB,XGB_score,XGB_score_LE
3,3,1,4,CCN1/C(=C/C2=[N+](C=CS2)CC3=CC=CC=C3)/S/C(=C/4...,JG-98,72547053,-6.6,0.719,32.0,-7.57911,5.604605,-7.641,-0.239


In [30]:
df1.to_csv('final_set/_site1.csv')